In [22]:
import pandas as pd
import re
import numpy as np

In [23]:
df = pd.read_csv('train.csv')
df.head()

,tweet,category
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0


In [24]:
df['og_tweet'] = df.loc[:, 'tweet']


In [25]:
def num_of_hash_tags(df):
    df['hashtags_count'] = df['tweet'].apply(lambda x: len([x for x in x.split() if x.startswith('#')]))

def num_of_words(df):
    df['word_count'] = df['tweet'].apply(lambda x : len(str(x).split(" ")))

def num_of_chars(df):
    df['char_count'] = df['tweet'].apply(lambda x: sum(list(map(len,x.split(' ')))))

def num_of_num(df):
    df['num_count'] = df['tweet'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))

def num_of_upper(df):
    df['upper_case'] = df['tweet'].apply(lambda x: len([x for x in x.split() if x.isupper()]))

def feature_extract(df):
    num_of_hash_tags(df)
    num_of_num(df)
    num_of_chars(df)
    num_of_upper(df)

df1 = df.copy().dropna()

feature_extract(df1)

df1.head()

,tweet,category,og_tweet,hashtags_count,num_count,char_count,upper_case
0,when modi promised “minimum government maximum...,-1.0,when modi promised “minimum government maximum...,0,0,178,0
1,talk all the nonsense and continue all the dra...,0.0,talk all the nonsense and continue all the dra...,0,0,55,0
2,what did just say vote for modi welcome bjp t...,1.0,what did just say vote for modi welcome bjp t...,0,0,95,0
3,asking his supporters prefix chowkidar their n...,1.0,asking his supporters prefix chowkidar their n...,0,0,179,0
4,answer who among these the most powerful world...,1.0,answer who among these the most powerful world...,0,0,67,0


In [26]:
def clean(raw):
    result = re.sub("<[a][^>]*>(.+?)</[a]>", 'Link.', raw)
    result = re.sub('&gt;', "", result) # greater than sign
    result = re.sub('&#x27;', "'", result) # apostrophe
#     result = re.sub('&quot;', '"', result) 
    result = re.sub('&#x2F;', ' ', result)
    result = re.sub('<p>', ' ', result) # paragraph tag
    result = re.sub('<i>', ' ', result) #italics tag
    result = re.sub('</i>', '', result) 
    result = re.sub('&#62;', '', result)
    result = re.sub("\n", '', result) # newline 
    return result

def deEmojify(x):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'', x)

def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    text = re.sub(r"[^a-zA-Z0-9?!.,]+", ' ', text)
    return text

def lower_case(df):
    df['tweet'] = df['tweet'].apply(lambda x: " ".join(x.lower() for x in x.split()))


In [27]:
def cleaning(df):
    df['tweet'] = df['tweet'].apply(clean)
    df['tweet'] = df['tweet'].apply(deEmojify)
    df['tweet'] = df['tweet'].apply(lambda x: remove_punct(x))
    lower_case(df)


In [28]:
import string

df2 = df1.copy()
cleaning(df2)
df2 = df2.dropna()
df2.head()

,tweet,category,og_tweet,hashtags_count,num_count,char_count,upper_case
0,when modi promised minimum government maximum ...,-1.0,when modi promised “minimum government maximum...,0,0,178,0
1,talk all the nonsense and continue all the dra...,0.0,talk all the nonsense and continue all the dra...,0,0,55,0
2,what did just say vote for modi welcome bjp to...,1.0,what did just say vote for modi welcome bjp t...,0,0,95,0
3,asking his supporters prefix chowkidar their n...,1.0,asking his supporters prefix chowkidar their n...,0,0,179,0
4,answer who among these the most powerful world...,1.0,answer who among these the most powerful world...,0,0,67,0


In [29]:
from sklearn.feature_extraction.text import CountVectorizer


In [30]:
data = df2['tweet']
data_labels = df2['category']


In [31]:
df2.head()


,tweet,category,og_tweet,hashtags_count,num_count,char_count,upper_case
0,when modi promised minimum government maximum ...,-1.0,when modi promised “minimum government maximum...,0,0,178,0
1,talk all the nonsense and continue all the dra...,0.0,talk all the nonsense and continue all the dra...,0,0,55,0
2,what did just say vote for modi welcome bjp to...,1.0,what did just say vote for modi welcome bjp t...,0,0,95,0
3,asking his supporters prefix chowkidar their n...,1.0,asking his supporters prefix chowkidar their n...,0,0,179,0
4,answer who among these the most powerful world...,1.0,answer who among these the most powerful world...,0,0,67,0


In [32]:
index = df2.index
df2['random_number'] = np.random.randn(len(index))
train = df2[df2['random_number'] <= 0.8]
test = df2[df2['random_number'] > 0.8]


In [33]:
# count vectorizer:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
train_matrix = vectorizer.fit_transform(train['tweet'])
test_matrix = vectorizer.transform(test['tweet'])


In [34]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()


In [35]:
X_train = train_matrix
X_test = test_matrix
y_train = train['category']
y_test = test['category']


In [36]:
lr.fit(X_train,y_train)


C:\Users\Subh Chaturvedi\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [37]:
predictions = lr.predict(X_test)


In [38]:
print(predictions)

[ 0.  1.  1. ...  0.  0. -1.]


In [39]:
# find accuracy, precision, recall:
from sklearn.metrics import confusion_matrix,classification_report
new = np.asarray(y_test)
confusion_matrix(predictions,y_test)


array([[ 6629,   131,   539],
       [  292, 11392,   341],
       [  581,   143, 14344]], dtype=int64)

In [40]:
print(classification_report(predictions,y_test))


              precision    recall  f1-score   support

        -1.0       0.88      0.91      0.90      7299
         0.0       0.98      0.95      0.96     12025
         1.0       0.94      0.95      0.95     15068

    accuracy                           0.94     34392
   macro avg       0.93      0.94      0.93     34392
weighted avg       0.94      0.94      0.94     34392



In [41]:
i=11
print((lr.predict(X_test[i]), test['og_tweet'].iloc[i], test['category'].iloc[i]))

(array([1.]), 'the three codes modi cracked give india huge foreign policy jumpstart via ', 1.0)


In [42]:
import pickle
with open('twittersentimentanalysis.pickle','wb') as f:
    pickle.dump(lr,f)

In [43]:
pickle.dump(vectorizer, open("vectorizer.pickle", "wb"))